## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [7]:
#python
print("{}+{}={}".format(1, 2, 3))

1+2=3


In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [9]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [10]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [11]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content="물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진을 많이 보여주는 것처럼요.\n\n2. **모델 설계**: 그 다음, 컴퓨터가 데이터를 이해하고 학습할 수 있도록 '모델'이라는 구조를 만듭니다. 이 모델은 사람의 뇌와 비슷하게 여러 층으로 이루어진 '신경망'일 수 있어요.\n\n3. **학습 과정**:\n   - 모델은 처음에는 아무것도 모르기 때문에, 예를 들어 고양이 사진인지 아닌지 구별하지 못해요.\n   - 모델이 사진을 보고 '이게 고양이일까 아니면 아니까'라고 추측합니다.\n   - 그리고 정답(이 사진은 고양이야!)과 비교해서, 틀렸다면 어디가 잘못됐는지 계산해요.\n   - 그 차이(오차)를 줄이기 위해 모델의 내부 값(가중치)을 조금씩 조정합니다.\n   \n4. **반복 학습**:\n   - 이 과정을 수천, 수만 번 반복하면서, 모델은 점점 더 정확하게 사진을 구별할 수 있게 돼요.\n   \n5. **완성**:\n   - 충분히 학습이 되면, 새로운 사진이 들어왔을 때도 고양이인지 아닌지 잘 맞출 수 있게 되는 거죠.\n\n요약하자면, 인공지능은 많은 예제와 반복 학습을 통해 '이것이 정답이다'라고 스스로 배우는 과정입니다. 이렇게 해서 점점 더 똑똑해지는 거예요!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 384, 'prompt_tokens': 22, 'total_tokens': 406, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_t

아래는 스트리밍을 출력하는 예시 입니다.

In [12]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배울 수 있도록 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진과 고양이 이름이 적힌 데이터가 있겠죠.

2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 학습할 수 있는 '모델'이라는 수학적 구조를 만듭니다. 이 모델은 사람의 뇌처럼 정보를 처리하는 방식과 비슷하게 만들어질 수 있어요.

3. **학습 과정**:
   - 모델은 처음에는 아무것도 모른 상태예요.
   - 데이터를 보여주면서, 모델이 예측한 결과와 실제 정답(예를 들어, 사진이 고양이인지 아닌지)을 비교해요.
   - 차이(오차)를 계산하고, 이 오차를 줄이기 위해 모델의 내부 값을 조금씩 조정해요. 이 과정을 여러 번 반복하면서 모델이 점점 더 정확하게 예측하게 되는 거죠.

4. **최적화**: 이때 '최적화 알고리즘'이라는 것이 사용돼서, 오차를 최소화하는 방향으로 모델의 내부 값을 조정하는 역할을 해요.

5. **학습 완료**: 여러 번 반복해서 학습하면, 모델은 새로운 데이터(예를 들어, 새로운 고양이 사진)도 잘 분류할 수 있게 돼요.

요약하자면, 인공지능 모델은 많은 데이터를 보고, 예측과 정답의 차이를 줄이기 위해 계속 조정하면서 점점 더 똑똑해지는 과정입니다.

### 출력파서(Output Parser)


In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [14]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [15]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

"물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진을 많이 보여주는 거죠.\n\n2. **모델 설계**: 그런 데이터를 이해하고 학습할 수 있도록 수학적인 구조(모델)를 만듭니다. 이 구조는 사람의 뇌와 비슷하게 여러 '뉴런'이 연결된 것처럼 생각할 수 있어요.\n\n3. **학습 과정**:\n   - **예측하기**: 모델은 처음에는 무작위로 예측을 해요. 예를 들어, 사진이 고양이인지 강아지인지 맞추려고 시도하는 거죠.\n   - **오차 계산**: 예측이 틀리면, 얼마나 틀렸는지(오차)를 계산해요.\n   - **수정하기**: 오차를 줄이기 위해 모델의 내부 값(가중치)을 조금씩 조정해요. 이 과정을 반복하면서 점점 더 정확한 예측을 하게 되는 거죠.\n\n4. **반복 학습**: 이 과정을 여러 번 반복하면서 모델은 점점 더 데이터를 잘 이해하게 되고, 새로운 데이터에 대해서도 올바르게 예측할 수 있게 돼요.\n\n요약하자면, 인공지능은 많은 데이터를 보고, 예측을 하고, 틀리면 고치면서 점점 더 똑똑해지는 과정입니다. 이렇게 해서 우리가 원하는 일을 할 수 있도록 학습하는 거예요!"

In [16]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진과 강아지 사진을 모으는 거죠.

2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 분류할 수 있도록 인공지능의 구조(모델)를 만듭니다. 이 구조는 사람의 뇌와 비슷하게 여러 층으로 이루어진 '신경망'이라고 부르기도 해요.

3. **학습 과정**:
   - 인공지능은 처음에는 아무것도 모르기 때문에, 사진이 고양이인지 강아지인지 맞추기 어렵습니다.
   - 그래서, 모델이 예측한 결과와 실제 정답(예를 들어, 사진이 고양이인지 강아지인지)을 비교해서 차이(오차)를 계산해요.
   - 이 차이를 줄이기 위해, 모델 내부의 '가중치'라는 값들을 조금씩 조정합니다.
   - 이 과정을 여러 번 반복하면서, 모델은 점점 더 정확하게 사진을 구분할 수 있게 됩니다.

4. **최적화**: 이때 '경사 하강법' 같은 방법을 사용해서, 오차를 가장 빠르게 줄일 수 있는 방향으로 가중치를 조정합니다.

5. **완성**: 충분히 학습이 되면, 인공지능은 새로운 사진이 들어왔을 때도 고양이인지 강아지인지 잘 맞출 수 있게 되는 거죠.

요약하자면, 인공지능은 많은 데이터를 보고, 예측과 정답의 차이를 줄이기 위해 내부 값을 계속 조정하면서 '배우는' 과정입니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [17]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [22]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate

# 공통 시스템 페르소나(권장: 시스템 메시지로 역할 고정)
SYSTEM = "당신은 10년차 영어 선생님입니다. 간결하지만 자연스러운 회화를 만들고, 학습 목적에 맞게 난이도를 조절하세요."

# (A) 기본형 — 현재 템플릿에 가까움
TEMPLATE_BASIC = """
주어진 상황에 맞는 영어 회화를 작성해 주세요.
[요구사항]
- 회화 턴 수: {turns}턴
- 난이도: {level} (예: 초급, 중급, 고급)
- 일상적으로 자연스럽고, 너무 과도한 슬랭은 지양

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# (B) 학습목표 명시형 — 타겟 표현/문법 강제
TEMPLATE_TARGET = """
학습 목표를 반영한 영어 회화를 작성해 주세요.
[요구사항]
- 회화 턴 수: {turns}턴
- 난이도: {level}
- 반드시 다음 요소를 최소 2회 사용: {targets}  # 예: present perfect, request politely, phrasal verb 'come up with'

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
- 사용된 표현 체크리스트:
"""

# (C) 평가 포함형 — 자기점검 루브릭 동봉
TEMPLATE_RUBRIC = """
상황에 맞는 영어 회화를 작성하고, 마지막에 간단한 자기평가 루브릭을 포함하세요.
[요구사항]
- 회화 턴 수: {turns}턴
- 난이도: {level}
- 어휘 레벨: CEFR {cefr} 기준
- 목표: {targets}

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
- 루브릭(자기점검 5점 척도 3항목):
"""

# (D) 구조화(JSON) 출력형 — 후처리/자동채점에 유리
TEMPLATE_JSON = """
다음 형식을 엄격히 지켜 JSON으로만 답하세요.
형식:
{{
  "dialogue": [{{"speaker":"A","en":"...","ko":"..."}}, ...],  # 총 {turns}턴
  "targets_covered": ["...", "..."],
  "notes": "학습 포인트 요약(한글)"
}}

[제약]
- 난이도: {level}, CEFR {cefr}
- 필수표현: {targets}

#상황:
{question}
"""


In [23]:
from langchain_teddynote.messages import stream_response
from langchain.schema.output_parser import StrOutputParser
# OpenAI 패키지 경로는 버전에 따라 다릅니다.
# (구버전) from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI  # 최신권장
parser = StrOutputParser()

def build_chain(template_str, model_name="gpt-4.1-nano", temperature=0.7):
    prompt = ChatPromptTemplate.from_messages([
        ("system", SYSTEM),
        ("user", template_str),
    ])
    # 모델 초기화 (버전에 따라 model 혹은 model_name 사용)
    try:
        llm = ChatOpenAI(model_name=model_name, temperature=temperature)
    except TypeError:
        llm = ChatOpenAI(model=model_name, temperature=temperature)
    return prompt | llm | parser


In [24]:
TEMPLATES = {
    "basic": TEMPLATE_BASIC,
    "target": TEMPLATE_TARGET,
    "rubric": TEMPLATE_RUBRIC,
    "json": TEMPLATE_JSON,
}

MODELS = [
    "gpt-4.1-nano",
    "gpt-4.1-mini",
    "gpt-4o-mini",
]

def run_experiment(question, level="중급", turns=6, targets="polite requests", cefr="B1",
                   template_key="basic", model_name="gpt-4.1-nano", temperature=0.7):
    chain = build_chain(TEMPLATES[template_key], model_name, temperature)
    inputs = {
        "question": question,
        "level": level,
        "turns": turns,
        "targets": targets,
        "cefr": cefr,
    }
    result = chain.invoke(inputs)
    print(f"\n=== [{template_key}] @ {model_name} (temp={temperature}) ===")
    print(result[:2000])  # 출력 너무 길면 앞부분만 미리보기
    return result

# 예시 실행
question = "카페에서 주문하고, 음료가 너무 달지 않게 해달라고 정중히 부탁하는 상황"
for m in MODELS:
    for t in ["basic", "target"]:
        run_experiment(question, level="중급", turns=6, targets="polite requests, 'could you ~'", template_key=t, model_name=m, temperature=0.6)



=== [basic] @ gpt-4.1-nano (temp=0.6) ===
영어 회화:
1. Customer: Hi, I’d like a large latte, please.
2. Barista: Sure, would you like any adjustments?
3. Customer: Yes, could you make it with less syrup? It’s a bit too sweet for me.
4. Barista: Of course. Would you like me to use less sugar or fewer sweeteners?
5. Customer: Just less syrup, please. Thank you.
6. Barista: No problem. I’ll have that ready for you shortly.

한글 해석:
1. 고객: 안녕하세요, 라지 라떼 하나 주세요.
2. 바리스타: 네, 조절 사항이 있으신가요?
3. 고객: 네, 시럽을 적게 넣어 주세요. 너무 달아서요.
4. 바리스타: 알겠습니다. 설탕을 적게 넣거나 다른 단맛 재료를 덜 넣어 드릴까요?
5. 고객: 그냥 시럽만 적게 넣어 주세요. 감사합니다.
6. 바리스타: 문제없어요. 곧 준비해 드리겠습니다.

=== [target] @ gpt-4.1-nano (temp=0.6) ===
영어 회화:  
Barista: Good afternoon! Welcome to Green Leaf Café. May I take your order?  
Customer: Hi! Yes, I’d like a caramel latte, please. Could you make it with less syrup? It tends to be too sweet for me.  
Barista: Of course! I could come up with a special blend to reduce the sweetness. Would you like any other modificatio

In [18]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [19]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
"Hello! I’d like to order a table for two, please."  
"Could I see the menu, please?"  
"I’ll have the grilled chicken salad and a glass of water."  
"Can I get the dressing on the side?"  
"Thank you. That will be all for now."

- 한글 해석:  
"안녕하세요! 2인용 테이블 예약하고 싶어요."  
"메뉴 좀 보여주시겠어요?"  
"그릴드 치킨 샐러드랑 물 한 잔 주세요."  
"드레싱은 따로 주시겠어요?"  
"감사합니다. 지금은 이것으로 주문 끝입니다."


In [20]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
"Hello, I would like to order a meal, please."  
"Hi, could I get the menu, please?"  
"Can I have the [specific dish] and a glass of water?"  

- 한글 해석:  
"안녕하세요, 음식을 주문하고 싶어요."  
"안녕하세요, 메뉴판 좀 주시겠어요?"  
"[특정 요리] 하나와 물 한 잔 주세요."

In [21]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
**Customer:** Hi, I’d like to order a large pepperoni pizza, please.  
**Employee:** Sure! Would you like curbside pickup or delivery?  
**Customer:** Delivery, please. How long will it take?  
**Employee:** It will be about 30 minutes. Can I have your address?  
**Customer:** Yes, it’s 123 Maple Street.  
**Employee:** Great. Your total is $15.99. Would you like to add any drinks or sides?  
**Customer:** No, that’s all. Thank you!  
**Employee:** Thank you! Your pizza will arrive soon.  

- 한글 해석:  
**손님:** 안녕하세요, 대형 페퍼로니 피자 하나 주문할게요.  
**직원:** 네! 포장하실 건가요 아니면 배달을 원하시나요?  
**손님:** 배달로 할게요. 얼마나 걸릴까요?  
**직원:** 약 30분 정도 걸립니다. 주소를 알려주시겠어요?  
**손님:** 네, 123 메이플 스트리트입니다.  
**직원:** 좋아요. 총 금액은 15.99달러입니다. 음료수나 사이드 메뉴도 추가하시겠어요?  
**손님:** 아니요, 그게 다예요. 감사합니다!  
**직원:** 감사합니다! 곧 피자가 배달될 거예요.